---
title: The Kinetics of a Diffusion-Controlled Reaction
authors:
  - name: Author Name
    email: example@cuny.edu
    affiliations:
      - ror: 00g2xk477
      - institution: CUNY – Hunter College
      - department: Chemistry
date: 2024-01-01
numbering:
  heading_2: true
  heading_3: true
---

Change the contents of the above block to reflect your name, email, and the current date. Additionally, be sure to change the file name to match the following format:  
*author's initials_title_date.ipynb*  
For example:  
*EF_Data Exploration_20210201.ipynb*  

When you've done that, go ahead and delete this block. 


## Purpose

Describe the purpose of this document. 

## Library import
We import all the required Python libraries

In [ ]:
# File handling
from pathlib import Path
import shutil
import re

# Data manipulation
import numpy as np
import scipy as sp

# Visualizations
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-v0_8-notebook')
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

## Data import
We retrieve all the required data for the analysis.

Prior to beginning this portion, make sure all of your data files have been uploaded to the same folder as this document (_i.e.,_ `~/357_fluorescence_analysis/`). 
If your data are still stored in a ZIP file, uncomment the bottom three lines in the cell below and fill in the relevant variables (`filename`, `data_folder`), then run the cell. If you already unzipped your files and have a folder full of fluorescence files, you'll need to set the variable `data_folder`, as later cells will use that value. 

In [ ]:
## Unzip your fluorescence data into a subfolder

# filename = Path('name of your zip file' + '.zip') # fill in the name of the file without the ZIP extension
# data_folder = Path('name of destination folder') # fill in a name for a subfolder in which to place the text files
# shutil.unpack_archive(filename, data_folder)

In [ ]:
# Create a list of all of the files in your new folder. 
# If your files end in something other than ".TXT", make that change here
data_files = list(data_folder.glob("*.TXT"))

# List filenames so we can figure out our column naming in the next step. 
for file in data_files:
    print(file) 

In [ ]:
# This is an example of a simple function with a "docstring". 
# When writing code, docstrings provide a simple way to document
# the purpose of a function within your code. When a user calls 
# `help(func)` on your function, the docstring will pull up and 
# tell the user how to proceed. 

# Create a simple function to import our fluorescence data and return 
# 1D arrays of wavelengths (wls) and absorption data (abs)
# Skip first 31 rows, as data doesn't start until line 32
# Default file from fluorometer uses tabs ('\t') for separators, also
# the default for the NumPy importers.
def fluor_import(fname):
    '''
    A simple function to import fluorescence data from the 
    Hitachi F-2700 spectrophotometer. 
    Returns a  pair of 1D arrays of wavelengths (wls) 
    and absorption data (abs)
    
    Parameters
    ----------
    fname: file, str, pathlib.Path
        File or filename to read. 

    Returns
    -------
    wls: ndarray 
        Wavelength values
    intst: ndarray 
        Fluorescence data
    '''
    
    data = np.genfromtxt(fname, skip_header=31, names=True)
    wls = data['nm']
    intst = data['Data']
    return wls, intst

Here, we use the `re` (Regular Expressions) library to search for patterns in text. Specifically, we're going to look for the percentage numbers in our filenames. This assumes your filenames have a two- or three-digit number representing the AN percent in your samples. There's a line included below that you'll need to _remove_ if you recorded the CBr<sub>4</sub> values instead. 

In [ ]:
# Create an empty dictionary to hold the contents of our assorted files
fluor_dict = {} 

for file in data_files:
    # Import the data using our function from the previous cell
    wls, intst = fluor_import(file)
    # Rename the dictionary entry to the percentage from your filename using
    # the `re` library (regular expressions)
    _val = re.findall(r'\d{1,3}', file.stem)[0]
    # Cast the key name to an integer so we can use it later for sorting and calculations
    _val = int(_val)
    # The next line turns the fluorophore percent into a quencher percent. 
    # Uncomment it if you recorded the AN value.
    # _val = 100-_val
    # Finally, add the data to our dictionary using the percentage as the key
    fluor_dict[int(_val)] = intst

# Print out the names of our keys to make sure they match the sample percentages
fluor_dict.keys()

In [ ]:
# Plot the spectra. Though we replaced `wls` each loop, they should 
# have all been the same, so we can use the current value of that variable. 
# Make sure your spectra decrease in intensity from 0%–100% CBr4
# For clarity and later use, we'll sort our concentrations from low to high
concs = np.array(sorted(fluor_dict.keys()))

for key in concs:
    plt.plot(wls, fluor_dict[key], label=key)

plt.legend()

## Data processing
This constitutes the core of the notebook. Feel free to further split this section into subsections.

In [ ]:
# Compile a sorted list of maximum intensities from each of our spectra
# We're using `np.sort()` instead of `sorted()` because it will be good
# practice for later to have a NumPy array as our output. 
intensities = []
for conc in concs:
    intensities.append(fluor_dict[conc].max())
    
intensities = np.array(intensities)

# The first line grabs the largest value from each spectrum, 
# then divides the largest value in that list by each maximum (calculating I_0/I)
intensities = intensities.max()/intensities 

for entry in zip(concs, intensities):
    print(entry)

In [ ]:
# Again, we'll define a simple function to help us out. 
def p2f(x): 
    '''Convert percentage to floating point numbers'''
    return float(x)/100

init_conc =  # Fill in the concentration (in molarity) of the original CBr4 solution

# The next line uses "list comprehension" to apply a function to each item in a list.
# We convert each string to a fractional value with p2f, 
# then multiply each fraction by the original concentration
concs = [(p2f(item) * init_conc) for item in concs]

plt.plot(concs, intensities, '.')
plt.xlabel('Concentration (M)')
plt.ylabel('$I_0/I$') 
plt.title('[Q] vs. $I_0/I$') # recall wrapping the input in `$` turns it into LaTeX math
plt.show()

### Collision radius via SES equation

In [ ]:
# Variable setup
# You'll need to create variables for the assorted values you'll use in 
# your calculations (e.g., tau_0, D, etc.)


In [ ]:
## SES: 
#    k = (8 * R_gas * T) / (3 * \eta)
#    k =  4 * \pi * N_A * radius * D / 1000
k_ses = 

# Can format numbers with the `:#.#g` syntax. `g` is an auto-formatter for 
# numbers that automatically trims and converts to scientific form if necessary. 
print(f"The reaction constant with the SES equation is {k_ses:g} 1/(M*s).") 

# Now calculate the radius
radius_ses = 
print(f"The reaction radius is {radius_ses:g} dm")

### Collision radius via SES equation

In [ ]:
# Variable setup
# Fill in appropriate variables for the SES equation. 

In [ ]:
## SES: k = (8 * R * T) / (3 * \eta)
k_ses = 
print(f"The reaction constant with the SES equation is {k_ses:g} 1/(M*s).") 

radius_ses = 
print(f"The reaction radius is {radius_ses:g} dm")

In [ ]:
# Now we need to plot a line based on the SES calculations

# We start by making a range of concentration values. 
# `linspace` is an ideal function as it makes evenly spaced 
# points between the start and stop values (50, by default). 
conc_data = np.linspace(0.000, init_conc)
ses_fit = tau_0 * k_ses * conc_data + 1

plt.plot(conc_data,ses_fit,'-',
        concs[:7], intensities[:7], ".")
plt.xlabel('Concentration (M)')
plt.ylabel('$I_0/I$')
plt.title('$I_0/I$ vs. [Q]')
plt.show()

### Collision radius via Stern-Vollmer relation (no transient term)

We're supposed to look at a linear fit for just the first few points. The `linregress()` function from `scipy.stats` works well for this (see the Plot Intro exercise for an example). Go ahead and plot a linear fit using different numbers of points from `concs` and `intensities` to see how the fit changes. 

In [ ]:
from scipy.stats import linregress

# Change this number to increase or decrease the number of points fit and plotted
pts = 4
fit_x = concs[:pts]
fit_y = intensities[:pts]

## Insert fitting routine here, use fit_x and fit_y as your parameters
sv_fit = linregress(

plt.plot(fit_x, fit_y, 'o', label='original data')
plt.plot(fit_x, (sv_fit.slope * fit_x + sv_fit.intercept), label='fitted line')
plt.legend()

Using the <code>\{eval\}\`py_variable\`</code> syntax we learned before, format and print the equation from your line as well as the standard error and $R^2$ value for the regression in a Markdown cell. 

For example, I can print out that $\sqrt{2}$ is {eval}`f'{np.sqrt(2):1.4f}'`. 

Feel free to replace the text in this cell with your text and fit parameters. 

In [ ]:
radius_fit = sv_fit.slope * 1000 / (4 * sp.constants.Avogadro  * np.pi * diffusion_hexane )
print(f"The reaction radius is {radius_fit:g} dm")

In [ ]:
# We'll compare the fit from the previous method to one using the curve_fit 
# routine from scipy.optimize. In your summary, comment on the differences 
# (and why they exist). The key is in the y-intercept…

def sv_func(conc, k_q):
    return k_q * tau_0 * conc + 1


from scipy.optimize import curve_fit

# Need to provide initial guess for parameter…
sv_coeff, sv_cov = curve_fit(sv_func, concs[:4], intensities[:4], p0=2e10)

sv_err = np.sqrt(np.diag(sv_cov))

print(sv_coeff, "\n", sv_err)
radius_sv = sv_coeff[0] * 1000 / (4 * sp.constants.Avogadro  * np.pi * diffusion_hexane )
print(f"The reaction radius is {radius_sv:g} dm")

### Collision radius via full Stern-Vollmer relation (with transient term)

In this section, you'll be fitting the full Stern-Vollmer relation and extracting the collision radius from that fit. You'll need to define functions for $a$, $b$, and $Y$, all of which are functions of $R$ and $[\text{CBr}_4]$. 

In [ ]:
def a_const(R, conc):
    return # fill in the equation for the "a" constant

def b_const(R, conc): 
    return # fill in the equation for the "b" constant

def y_const(R, conc):
    # fill in the equation for the "Y" constant
    # you must input `a` and `b` as 
    # `a_const(R, conc)` and `b_const(R, cons)`
    # You'll find it helpful to do this one in multiple steps
    return 

In [ ]:
# It's worth playing with values of R and conc to see how they affect Y
# Experiment with them here. 
print(y_const(5e-8, 1.5e-2))

In [ ]:
def intensity(conc, R):
    return # fill in the equation for the intensity (I_0/I) as a function
           # of conc, R, and y_const(conc, R)

# The function requires a guess for all fitted parameters. 
# We'll use the value previously calculated with the SES equation. 
# It needs to be converted from dm to cm.
sv_coeff_2 , sv_cov_2 = curve_fit(intensity, 
                                  concs, 
                                  intensities, 
                                  p0=[0.1 * radius_ses] 
                                 )

sv_err_2 = np.sqrt(np.diag(sv_cov_2))

print(f"R = {sv_coeff_2[0]:g} ± {sv_err_2[0]:.2e} cm")

# Create a plot that shows your fit compared to the experimental data


## Results
Describe and comment the most important results.


## References
We report here relevant references:
1. author1, article1, journal1, year1, url1
2. author2, article2, journal2, year2, url2